# Trucks only problem #

Install necessary packages

In [102]:
from gurobipy import Model,GRB,LinExpr,quicksum
import numpy as np
from scipy.spatial import distance
import os
import socket
from load_dataset import Dataset

For Ugo's laptop

In [103]:
# Define the node name or another identifier of your laptop
my_laptop_node = 'Ugos-MacBook-Pro.local'

# Get the current system's node name using socket.gethostname()
current_node = socket.gethostname()

if current_node == my_laptop_node:
    # Set the environment variable for Gurobi license file
    os.environ["GRB_LICENSE_FILE"] = "/Users/ugomunzi/gurobi/licenses/gurobi.lic"
    print("Gurobi license path set for Ugo's MacBook Pro.")
else:
    print("Not Ugo's MacBook Pro, using default or no specific license settings.")

Not Ugo's MacBook Pro, using default or no specific license settings.


Define model parametres

In [104]:
## MODEL PARAMETERS ##
W_T = 1500 #empty weight truck [kg]
Q_T = 1000 #load capacity of trucks [kg]
W_D = 25 #empty weight drone [kg]
Q_D = 5 #load capacity of drones [kg]
C_T = 25 #travel cost of trucks per unit distance [monetary unit/km]
C_D = 1 #travel cost of drones per unit distance [monetary unit/km]
C_B = 500 #basis cost of using a truck equipped with a drone [monetary unit]
E = 0.5 #maximum endurance of empty drones [hours]
S_T = 60 #average travel speed of the trucks [km/h]
S_D = 65 #average travel speed of the drones [km/h]

Define Big M constant

In [105]:
M = 500 #big M constant for big M method

Load Dataset using load_dataset.py

In [106]:
## LOAD DATASET ##
current_dir = os.getcwd()
# Select which data folder to use
data_subfolder = '0.3'
data_subfoldercopy = '0.3_copy'
data_num_nodes = '40'
data_area = '20'

data_file_name = f'{data_num_nodes}_{data_area}_{data_subfoldercopy}'
dataset_path = f'dataset/{data_subfolder}/{data_file_name}.txt'
output_file_path = os.path.join(current_dir, data_file_name + '_solution.sol')#used to save solution file

dataset = Dataset(dataset_path)



Pre-processing

In [107]:
## FUNCTIONS ##
def get_manhattan_distance(data):
    """
    Returns a dictionary with manhattan distances between all nodes in dataset
    """
    distance_dict = {}
    for node1 in data.keys():
        for node2 in data.keys():
            distance_dict[node1, node2] = distance.cityblock([data[node1]['X'], data[node1]['Y']], [data[node2]['X'], data[node2]['Y']])
    return distance_dict

def get_time_dict(data, S_T, distance_dict):
    """
    Returns a dictionary with travel times between all nodes in dataset
    """
    time_dict = {}
    for node1 in data.keys():
        for node2 in data.keys():
            time_dict[node1, node2] = distance_dict[node1, node2] / S_T
    return time_dict


num_trucks = 2
distance_dict = get_manhattan_distance(dataset.data)
time_dict = get_time_dict(dataset.data, S_T, distance_dict)

#definitions of N_0, N and N_plus follow from paper
N = list(dataset.data.keys()) #set of nodes with depot at start
N_customers = N.copy()
N_customers.remove('D0')
V = [f'V{i}' for i in range(1, num_trucks+1)] #set of trucks

Define the model

In [108]:
# Define your data here
# Assume distance_dict, C_T, C_B, N, V, N_customers, time_dict, M, Q_T are all defined as per your problem context

# Create a new model
model = Model("Truck_Routing")

# Define decision variables
x = model.addVars(V, [(i, j) for i in N for j in N if i != j], lb=0, ub=1, vtype=GRB.BINARY, name='x')
y = model.addVars(V, lb=0, ub=1, vtype=GRB.BINARY, name='y')
t = model.addVars(V, N, lb=0, vtype=GRB.CONTINUOUS, name='t')

# Objective function (minimize cost both due to transportation and base cost of using truck if active)
cost_obj = quicksum(C_T * distance_dict[i,j] * x[v,i,j] for i in N for j in N if i != j for v in V) + quicksum(C_B * y[v] for v in V)
model.setObjective(cost_obj, GRB.MINIMIZE)

model.update()

Constraint 1: Each customer is visited by exactly one truck

In [109]:
# Each customer is visited by exactly one truck
constraints = {}
# Loop over each customer
for customer in N_customers:
    # Initialize the sum for the current customer
    sum_for_current_customer = 0

    # Loop over each vehicle
    for vehicle in V:
        # Loop over each node
        for node in N:
            # Skip if customer is equal to node
            if customer != node:
                # Add the variable to the sum
                sum_for_current_customer += x[vehicle, node, customer]

    # The sum for the current customer must be equal to 1
    constraints[node] = sum_for_current_customer == 1

    # Add the constraints to the model
    model.addConstr(constraints[node], name='Each_customer_visited_once')

Constraint 2: Each depot must be visited exactly once

In [110]:
# Each truck must leave the depot
# y - active
# 'D0' - depot
# Loop over each vehicle
for vehicle in V:
    # Initialize the sum for the current vehicle
    sum_for_current_vehicle = quicksum(x[vehicle, 'D0', customer] for customer in N_customers)
    
    #### THE ACTIVE PART IS WRONG, NEED TO FIX THIS !!!
    # Add the constraint to the model
    #model.addConstr(sum_for_current_vehicle == 1, name=f'Truck_leaves_depot_{vehicle}')

    # Add the constraint to the model
    model.addGenConstrIndicator(y[vehicle], 1, sum_for_current_vehicle == 1, name=f'Truck_leaves_depot_if_active_{vehicle}')



Update, tune and run the model

In [111]:
# Update the model to integrate new variables
model.update()

# Write model to a file
model.write('TruckonlySimple.lp')

# Tune solver parameters
model.tune()

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print('Optimal objective: %g' % model.objVal)
    for v in model.getVars():
        if v.x > 0:
            print('%s: %g' % (v.varName, v.x))
else:
    print('Optimization was stopped with status %d' % model.status)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Start tuning.

Solving model using baseline parameter set with TimeLimit=3600s

-------------------------------------------------------------------------------

Testing candidate parameter set 1...

	Default parameters

Solving Truck_Routing with random seed #1 ... MIP gap 0.01%
Solving Truck_Routing with random seed #2 ... MIP gap 0.01%
Solving Truck_Routing with random seed #3 ... MIP gap 0.01%

Summary candidate parameter set 1 
 # Name              0        1        2      Avg  Std Dev      Max
 0 Truck_Rou     0.00s    0.00s    0.00s    0.00s     0.00    0.00s

Setting total tuning time limit to 0s
(set the TuneTimeLimit parameter to choose a different value)

-------------------------------------------------------------------------------

Post-processing

In [112]:
# Extract and store the solution
solution = {var.varName: var.x for var in model.getVars()}

# Print all routes for each vehicle
for vehicle in V:
    print(f'Vehicle {vehicle}:')
    for node_from in N:
        for node_to in N:
            if node_from != node_to:
                var_name = f'x[{vehicle},{node_from},{node_to}]'
                if var_name in solution and solution[var_name] >= 0.99:
                    print(f'{node_from} -> {node_to}')
    print()



Vehicle V1:
C2 -> C3
C3 -> C2

Vehicle V2:
D0 -> C6
C1 -> C5
C5 -> C1
C8 -> C9
C8 -> C10
C9 -> C7
C10 -> C4
C10 -> C8



Old post-processing

In [113]:
#exctract active vehicles
active_vehicles = [v for v in V if solution[f'y[{v}]'] >= 0.99]
#extract routes
active_routes = {}
for v in active_vehicles:
    active_routes[v] = [i for i in N if solution[f'x[{v},{i},D0]'] >= 0.99] #start with depot
    while active_routes[v][-1] != 'D0':
        for j in N:
            if solution[f'x[{v},{active_routes[v][-1]},{j}]'] >= 0.99:
                active_routes[v].append(j)
                break

#retrieve timestamps of customer visits
timestamps = {}
for v in active_vehicles:
    timestamps[v] = {}
    for i in N_customers:
        timestamps[v][i] = solution[f't[{v},{i}]']

#print all solution variables which have value of 1
print([var for var in solution.keys() if solution[var] >=0.9])
print(active_routes)

['x[V1,C2,C3]', 'x[V1,C3,C2]', 'x[V2,D0,C6]', 'x[V2,C1,C5]', 'x[V2,C5,C1]', 'x[V2,C8,C9]', 'x[V2,C8,C10]', 'x[V2,C9,C7]', 'x[V2,C10,C4]', 'x[V2,C10,C8]']
{}
